In this notebook we will demonstrate various CNN and RNN models for the task of intent detection on the ATIS dataset. 

In [25]:

#making the necessary imports
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model, Sequential
from keras.initializers import Constant

from sklearn.preprocessing import LabelEncoder

import random
random.seed(0)#for reproducability of results

import pandas as pd
import numpy as np

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading the training data

In [27]:
import os
os.getcwd()

'/content'

In [28]:
import pandas as pd
import numpy as np

def get_data(filename):
    df = pd.read_csv(filename,delim_whitespace=True,names=['word','label'])
    beg_indices = list(df[df['word'] == 'BOS'].index)+[df.shape[0]]
    sents,labels,intents = [],[],[]
    for i in range(len(beg_indices[:-1])):
        sents.append(df[beg_indices[i]+1:beg_indices[i+1]-1]['word'].values)
        labels.append(df[beg_indices[i]+1:beg_indices[i+1]-1]['label'].values)
        intents.append(df.loc[beg_indices[i+1]-1]['label'])    
    return np.array(sents, dtype=object), np.array(labels, dtype=object), np.array(intents, dtype=object)

def get_data2(filename):
    with open(filename) as f:
        contents = f.read()
    sents,labels,intents = [],[],[]
    for line in contents.strip().split('\n'):
        words,labs = [i.split(' ') for i in line.split('\t')]
        sents.append(words[1:-1])
        labels.append(labs[1:-1])
        intents.append(labs[-1])
    return np.array(sents, dtype=object), np.array(labels, dtype=object), np.array(intents, dtype=object)

read_method = {'/content/drive/My Drive/Data/NLP/Practical_NLP_Oreilly/Chap6/data2/atis-2.dev.w-intent.iob':get_data,
               '/content/drive/My Drive/Data/NLP/Practical_NLP_Oreilly/Chap6/data2/atis.train.w-intent.iob':get_data2,
               '/content/drive/My Drive/Data/NLP/Practical_NLP_Oreilly/Chap6/data2/atis.test.w-intent.iob':get_data,
              '/content/drive/My Drive/Data/NLP/Practical_NLP_Oreilly/Chap6/data2/atis-2.train.w-intent.iob':get_data2}

def fetch_data(fname):
    func = read_method[fname]
    return func(fname)

In [29]:
#utils is included in this repository'c Ch6 folder under folder name "Data"
#from Data.utils import fetch_data, read_method
sents,labels,intents = fetch_data('/content/drive/My Drive/Data/NLP/Practical_NLP_Oreilly/Chap6/data2/atis.train.w-intent.iob')
print(len(sents),len(labels),len(intents))

4978 4978 4978


In [30]:
sents

array([list(['i', 'want', 'to', 'fly', 'from', 'boston', 'at', '838', 'am', 'and', 'arrive', 'in', 'denver', 'at', '1110', 'in', 'the', 'morning']),
       list(['what', 'flights', 'are', 'available', 'from', 'pittsburgh', 'to', 'baltimore', 'on', 'thursday', 'morning']),
       list(['what', 'is', 'the', 'arrival', 'time', 'in', 'san', 'francisco', 'for', 'the', '755', 'am', 'flight', 'leaving', 'washington']),
       ...,
       list(['which', 'airlines', 'fly', 'into', 'and', 'out', 'of', 'denver']),
       list(['does', 'continental', 'fly', 'from', 'boston', 'to', 'san', 'francisco', 'with', 'a', 'stop', 'in', 'denver']),
       list(['is', 'there', 'a', 'delta', 'flight', 'from', 'denver', 'to', 'san', 'francisco'])],
      dtype=object)

In [31]:
labels

array([list(['O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'O', 'B-toloc.city_name', 'O', 'B-arrive_time.time', 'O', 'O', 'B-arrive_time.period_of_day']),
       list(['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'O', 'B-depart_date.day_name', 'B-depart_time.period_of_day']),
       list(['O', 'O', 'O', 'B-flight_time', 'I-flight_time', 'O', 'B-fromloc.city_name', 'I-fromloc.city_name', 'O', 'O', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'B-fromloc.city_name']),
       ..., list(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-city_name']),
       list(['O', 'B-airline_name', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name', 'O', 'O', 'O', 'O', 'B-stoploc.city_name']),
       list(['O', 'O', 'O', 'B-airline_name', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name'])],
      dtype=object)

In [32]:
intents

array(['atis_flight', 'atis_flight', 'atis_flight_time', ...,
       'atis_airline', 'atis_flight', 'atis_flight'], dtype=object)

In [33]:
train_sentences = [" ".join(i) for i in sents]
train_sentences[:10]

['i want to fly from boston at 838 am and arrive in denver at 1110 in the morning',
 'what flights are available from pittsburgh to baltimore on thursday morning',
 'what is the arrival time in san francisco for the 755 am flight leaving washington',
 'cheapest airfare from tacoma to orlando',
 'round trip fares from pittsburgh to philadelphia under 1000 dollars',
 'i need a flight tomorrow from columbus to minneapolis',
 'what kind of aircraft is used on a flight from cleveland to dallas',
 'show me the flights from pittsburgh to los angeles on thursday',
 'all flights from boston to washington',
 'what kind of ground transportation is available in denver']

In [34]:
train_texts = train_sentences
train_labels= intents.tolist()
train_labels[:5]

['atis_flight',
 'atis_flight',
 'atis_flight_time',
 'atis_airfare',
 'atis_airfare']

In [35]:
set(train_labels)

{'atis_abbreviation',
 'atis_aircraft',
 'atis_aircraft#atis_flight#atis_flight_no',
 'atis_airfare',
 'atis_airfare#atis_flight_time',
 'atis_airline',
 'atis_airline#atis_flight_no',
 'atis_airport',
 'atis_capacity',
 'atis_cheapest',
 'atis_city',
 'atis_distance',
 'atis_flight',
 'atis_flight#atis_airfare',
 'atis_flight_no',
 'atis_flight_time',
 'atis_ground_fare',
 'atis_ground_service',
 'atis_ground_service#atis_ground_fare',
 'atis_meal',
 'atis_quantity',
 'atis_restriction'}

In [36]:
vals = []

for i in range(len(train_labels)):
    if "#" in train_labels[i]:
        vals.append(i)
        
for i in vals[::-1]:
    train_labels.pop(i)
    train_texts.pop(i)

print ("Number of training sentences :",len(train_texts))
print ("Number of unique intents :",len(set(train_labels)))

for i in zip(train_texts[:5], train_labels[:5]):
    print(i)

Number of training sentences : 4952
Number of unique intents : 17
('i want to fly from boston at 838 am and arrive in denver at 1110 in the morning', 'atis_flight')
('what flights are available from pittsburgh to baltimore on thursday morning', 'atis_flight')
('what is the arrival time in san francisco for the 755 am flight leaving washington', 'atis_flight_time')
('cheapest airfare from tacoma to orlando', 'atis_airfare')
('round trip fares from pittsburgh to philadelphia under 1000 dollars', 'atis_airfare')


Loading the test data

In [37]:
#from Data.utils import fetch_data, read_method

sents,labels,intents = fetch_data('/content/drive/My Drive/Data/NLP/Practical_NLP_Oreilly/Chap6/data2/atis.test.w-intent.iob')

test_sentences = [" ".join(i) for i in sents]

test_texts = test_sentences
test_labels = intents.tolist()

new_labels = set(test_labels) - set(train_labels)

vals = []

for i in range(len(test_labels)):
    if "#" in test_labels[i]:
        vals.append(i)
    elif test_labels[i] in new_labels:
        print(test_labels[i])
        vals.append(i)
        
for i in vals[::-1]:
    test_labels.pop(i)
    test_texts.pop(i)

print ("Number of testing sentences :",len(test_texts))
print ("Number of unique intents :",len(set(test_labels)))

for i in zip(test_texts[:5], test_labels[:5]):
    print(i)

atis_day_name
atis_day_name
Number of testing sentences : 876
Number of unique intents : 15
('i would like to find a flight from charlotte to las vegas that makes a stop in st. louis', 'atis_flight')
('on april first i need a ticket from tacoma to san jose departing before 7 am', 'atis_airfare')
('on april first i need a flight going from phoenix to san diego', 'atis_flight')
('i would like a flight traveling one way from phoenix to san diego on april first', 'atis_flight')
('i would like a flight from orlando to salt lake city for april first on delta airlines', 'atis_flight')


In [38]:
set(test_labels)

{'atis_abbreviation',
 'atis_aircraft',
 'atis_airfare',
 'atis_airline',
 'atis_airport',
 'atis_capacity',
 'atis_city',
 'atis_distance',
 'atis_flight',
 'atis_flight_no',
 'atis_flight_time',
 'atis_ground_fare',
 'atis_ground_service',
 'atis_meal',
 'atis_quantity'}

Pre-Processing

In [39]:
MAX_SEQUENCE_LENGTH = 300
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.3

Su dung Toeknizer (max 20,000 tu) de hoc toan bo tu trong train_texts, sau do se ma hoa tung tu trong cau theo word index (sap xep theo thu tu pho bien nhat)

In [40]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts) #Converting text to a vector of word indexes
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 897 unique tokens.


In [41]:
word_index

{'0900': 846,
 '1': 308,
 '10': 148,
 '100': 539,
 '1000': 227,
 '1017': 844,
 '1020': 805,
 '1024': 638,
 '1026': 682,
 '1030': 656,
 '1039': 694,
 '1045': 758,
 '1055': 771,
 '1059': 731,
 '106': 664,
 '1083': 845,
 '11': 379,
 '110': 722,
 '1100': 672,
 '1110': 530,
 '1115': 446,
 '1130': 657,
 '1133': 851,
 '1145': 582,
 '1158': 739,
 '12': 176,
 '1200': 833,
 '1205': 780,
 '1209': 871,
 '1220': 715,
 '1222': 572,
 '1230': 836,
 '124': 683,
 '1245': 447,
 '1288': 645,
 '1291': 501,
 '130': 747,
 '1300': 873,
 '137338': 828,
 '139': 670,
 '150': 721,
 '1500': 868,
 '1505': 813,
 '1600': 706,
 '163': 890,
 '1700': 548,
 '1765': 553,
 '1850': 812,
 '19': 668,
 '1940': 881,
 '1991': 258,
 '1992': 450,
 '1993': 809,
 '2': 228,
 '200': 697,
 '201': 567,
 '21': 502,
 '210': 578,
 '2100': 461,
 '212': 855,
 '2134': 550,
 '2153': 659,
 '217': 590,
 '225': 738,
 '229': 841,
 '230': 434,
 '257': 646,
 '269': 612,
 '270': 523,
 '271': 732,
 '279': 827,
 '281': 397,
 '296': 513,
 '297': 700,
 '

In [42]:
train_sequences[:3]

[[18, 70, 1, 38, 2, 9, 67, 402, 86, 16, 79, 15, 12, 67, 530, 15, 4, 35],
 [6, 3, 26, 58, 2, 19, 1, 22, 5, 77, 35],
 [6, 20, 4, 403, 180, 15, 11, 14, 37, 4, 531, 86, 8, 33, 32]]

In [43]:
le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)
print(train_labels.shape)
print(np.unique(train_labels))
train_labels

(4952,)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


array([ 9,  9, 11, ...,  3,  9,  9])

In [44]:
#Converting this to sequences to be fed into neural network. Max seq. len is 1000 as set earlier
 #initial padding of 0s, until vector is of size MAX_SEQUENCE_LENGTH
trainvalid_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
trainvalid_labels = to_categorical(train_labels)
print(trainvalid_data.shape)
trainvalid_data

(4952, 300)


array([[  0,   0,   0, ...,  15,   4,  35],
       [  0,   0,   0, ...,   5,  77,  35],
       [  0,   0,   0, ...,   8,  33,  32],
       ...,
       [  0,   0,   0, ..., 230,  29,  12],
       [  0,   0,   0, ..., 109,  15,  12],
       [  0,   0,   0, ...,   1,  11,  14]], dtype=int32)

In [58]:
print(trainvalid_labels.shape)
trainvalid_labels

(4952, 17)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [45]:
print(np.unique(test_labels))
test_labels

[ 0  1  2  3  4  5  7  8  9 10 11 12 13 14 15]


array([ 9,  2,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 13,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 14, 14,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  4,  9,  9,  9,  9,  9,  9,  9,  9,  9,  2,  3,
       11,  9,  9,  7,  3,  9,  9,  9,  9,  9,  9,  9,  9,  9, 13,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  3,  9,  9,  9,  9,  9,  9, 13, 12, 12,
        9, 13,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  3,  9,
        9,  9,  9,  9,  9,  9,  9, 15, 15, 15,  9,  9,  4,  9,  9,  9,  2,
        2,  9,  0, 14,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        8,  8,  8,  8, 12, 12, 12, 12,  3,  9,  9,  2,  1,  9,  2,  9,  9,
        9,  9,  9,  3,  3,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  0,  8,  8,  8

In [53]:
len(set(train_labels))

17

In [49]:
trainvalid_labels.shape

(4952, 17)

In [56]:
len(set(test_labels))

15

In [47]:
test_labels.shape

(876,)

In [46]:
#DNN change test_labels = test_labels_new:
# Trainlabel co 17 unique label, hot encoding thanh 4952x17, ap dung cung shape encoding nay cho test label (15 unique thoi)
test_labels_new = to_categorical(np.asarray(test_labels), num_classes= trainvalid_labels.shape[1])
print(test_labels_new.shape)
test_labels_new

(876, 17)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [59]:
# split the training data into a training set and a validation set
indices = np.arange(trainvalid_data.shape[0])
np.random.shuffle(indices)
trainvalid_data = trainvalid_data[indices]
trainvalid_labels = trainvalid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * trainvalid_data.shape[0])
x_train = trainvalid_data[:-num_validation_samples]
y_train = trainvalid_labels[:-num_validation_samples]
x_val = trainvalid_data[-num_validation_samples:]
y_val = trainvalid_labels[-num_validation_samples:]
#This is the data we will use for CNN and RNN training
print('Splitting the train data into train and valid is done')

Splitting the train data into train and valid is done


In [65]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(3467, 300)
(3467, 17)
(1485, 300)
(1485, 17)


In [61]:

x_train

(3467, 300)


array([[  0,   0,   0, ..., 172,  82,  63],
       [  0,   0,   0, ..., 171, 102,  36],
       [  0,   0,   0, ...,  12,   1,  24],
       ...,
       [  0,   0,   0, ...,   9,   1,  24],
       [  0,   0,   0, ...,  19,  16,  17],
       [  0,   0,   0, ...,   9,   1,  19]], dtype=int32)

In [64]:

y_train

(3467, 17)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [68]:
word_index.items()

dict_items([('to', 1), ('from', 2), ('flights', 3), ('the', 4), ('on', 5), ('what', 6), ('me', 7), ('flight', 8), ('boston', 9), ('show', 10), ('san', 11), ('denver', 12), ('a', 13), ('francisco', 14), ('in', 15), ('and', 16), ('atlanta', 17), ('i', 18), ('pittsburgh', 19), ('is', 20), ('dallas', 21), ('baltimore', 22), ('all', 23), ('philadelphia', 24), ('like', 25), ('are', 26), ('list', 27), ('airlines', 28), ('of', 29), ('that', 30), ('between', 31), ('washington', 32), ('leaving', 33), ('please', 34), ('morning', 35), ('pm', 36), ('for', 37), ('fly', 38), ('would', 39), ('fare', 40), ('first', 41), ('oakland', 42), ('there', 43), ('wednesday', 44), ('after', 45), ('ground', 46), ("i'd", 47), ('you', 48), ('trip', 49), ('does', 50), ('transportation', 51), ('cheapest', 52), ('class', 53), ('need', 54), ('round', 55), ('city', 56), ('arriving', 57), ('available', 58), ('with', 59), ('have', 60), ('before', 61), ('which', 62), ('afternoon', 63), ('american', 64), ('one', 65), ('give'

In [67]:
print('Preparing embedding matrix.')

# first, build index mapping words in the embeddings set
# to their embedding vector

# Download GloVe 6B from here: https://nlp.stanford.edu/projects/glove/
#BASE_DIR = 'Data'
#GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
path='/content/drive/My Drive/Data/NLP/glove.6B.100d.txt'
embeddings_index = {}
with open(path, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))
#print(embeddings_index["google"])

# prepare embedding matrix - rows are the words from word_index, columns are the embeddings of that word from glove.
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load these pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
print("Preparing of embedding matrix is done")

Preparing embedding matrix.
Found 400000 word vectors in Glove embeddings.
Preparing of embedding matrix is done


In [70]:
print(x_train.shape)
print(y_train.shape)

(3467, 300)
(3467, 17)


In [72]:
print('Define a 1D CNN model.')

cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(trainvalid_labels[0]), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

cnnmodel.summary()

#Train the model. Tune to validation set. 
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=1, validation_data=(x_val, y_val))


Define a 1D CNN model.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 296, 128)          64128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 55, 128)           82048     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 11, 128)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 7, 128)            82048     
_________________________________________________________________
global_max_pooling1d_1 (Glob (N

In [88]:
y_predict=cnnmodel.predict(test_data)
print(y_predict.shape)
y_predict

(876, 17)


array([[2.4512003e-03, 1.1557563e-03, 7.9398407e-03, ..., 8.8100336e-05,
        5.0305965e-04, 6.0530852e-05],
       [2.5701636e-02, 1.3447735e-02, 5.1745933e-02, ..., 2.3322753e-03,
        7.5318613e-03, 1.6836181e-03],
       [1.2119862e-02, 6.1163111e-03, 2.9348634e-02, ..., 8.0645038e-04,
        3.1490212e-03, 5.6940404e-04],
       ...,
       [1.6184507e-02, 8.3069159e-03, 3.6921248e-02, ..., 1.2111645e-03,
        4.4140299e-03, 8.6560915e-04],
       [1.0893756e-02, 5.5145561e-03, 2.6579374e-02, ..., 6.9075776e-04,
        2.7596392e-03, 4.8628321e-04],
       [1.3420280e-02, 6.7542442e-03, 3.2132156e-02, ..., 9.3006023e-04,
        3.5091774e-03, 6.6274719e-04]], dtype=float32)

In [97]:
cnnmodel.predict_classes(test_data)

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,

In [96]:
test_labels

array([ 9,  2,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 13,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 14, 14,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  4,  9,  9,  9,  9,  9,  9,  9,  9,  9,  2,  3,
       11,  9,  9,  7,  3,  9,  9,  9,  9,  9,  9,  9,  9,  9, 13,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  3,  9,  9,  9,  9,  9,  9, 13, 12, 12,
        9, 13,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  3,  9,
        9,  9,  9,  9,  9,  9,  9, 15, 15, 15,  9,  9,  4,  9,  9,  9,  2,
        2,  9,  0, 14,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        8,  8,  8,  8, 12, 12, 12, 12,  3,  9,  9,  2,  1,  9,  2,  9,  9,
        9,  9,  9,  3,  3,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  0,  8,  8,  8

In [93]:
test_labels_new

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [94]:
print(test_labels_new[0])
print(y_predict[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[2.4512003e-03 1.1557563e-03 7.9398407e-03 3.1934285e-03 5.3925969e-04
 3.9139597e-04 5.1026462e-05 2.5046832e-04 6.7450514e-05 9.7949743e-01
 2.7148463e-04 9.4396400e-04 3.0375275e-04 2.2918175e-03 8.8100336e-05
 5.0305965e-04 6.0530852e-05]


In [95]:
print(test_labels_new[1])
print(y_predict[1])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.02570164 0.01344774 0.05174593 0.02511717 0.00547589 0.00561662
 0.00148131 0.00471546 0.0021219  0.809255   0.00413144 0.01166215
 0.00456476 0.02341527 0.00233228 0.00753186 0.00168362]


In [75]:
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels_new)
print('Test accuracy with CNN:', acc)

28/28 [==============================] - 1s 31ms/step - loss: 0.9768 - acc: 0.7580
Test accuracy with CNN: 0.7579908967018127


In [98]:
MAX_NUM_WORDS

20000

In [76]:
print("Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings")
cnnmodel = Sequential()
cnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(trainvalid_labels[0]), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

cnnmodel.summary()

#Train the model. Tune to validation set. 
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=1, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels_new)
print('Test accuracy with CNN:', acc)


Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 128)         82048     
_________

In [77]:
print("Defining and training an LSTM model, training embedding layer on the fly")

#modified from: 

rnnmodel = Sequential()
rnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel.summary()

print('Training the RNN')
rnnmodel.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_val, y_val))
score, acc = rnnmodel.evaluate(test_data, test_labels_new,
                            batch_size=32)
print('Test accuracy with RNN:', acc)


Defining and training an LSTM model, training embedding layer on the fly
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 17)                2193      
Total params: 2,693,777
Trainable params: 2,693,777
Non-trainable params: 0
_________________________________________________________________
Training the RNN
28/28 [==============================] - 3s 97ms/step - loss: 0.1142 - accuracy: 0.7215
Test accuracy with RNN: 0.7214611768722534


In [78]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

#modified from: 

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel2.summary()

print('Training the RNN')
rnnmodel2.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels_new,
                            batch_size=32)
print('Test accuracy with RNN:', acc)


Defining and training an LSTM model, using pre-trained embedding layer
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_7 (Dense)              (None, 17)                2193      
Total params: 209,241
Trainable params: 119,441
Non-trainable params: 89,800
_________________________________________________________________
Training the RNN
28/28 [==============================] - 3s 92ms/step - loss: 0.1003 - accuracy: 0.7215
Test accuracy with RNN: 0.7214611768722534
